In [6]:
from elasticsearch import Elasticsearch, helpers
import time
import os

# Connect to Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

In [7]:
# Delete the index if it exists
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

In [8]:
# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text",
                      "analyzer": "italian",
                      "search_analyzer": "italian"},
            "content": {"type": "text", 
                        "analyzer": "italian",
                        "search_analyzer": "italian"}
        }
    }
}

es.indices.create(index='index_recipes', body=mapping)
print("Index 'index_recipes' with mapping created successfully.")

Index 'index_recipes' with mapping created successfully.


In [9]:
def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Remove file extension for title
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Bulk index the documents with only one call to Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documents indexed in the '{index_name}' index.")
    else:
        print("No .txt files found in the directory.")

In [10]:
# Index documents from the 'files' directory
before = time.time()
bulk_index("files", "index_recipes")
after = time.time()
print(f"Indexing took {after - before} seconds")

5939 documents indexed in the 'index_recipes' index.
Indexing took 1.5928459167480469 seconds


In [11]:
def search(query):
   res = es.search(index='index_recipes', body=query)
   print("Search Results:")
   for i, hit in enumerate(res['hits']['hits']):
    res_doc = hit['_source']
    print(f"{i+1}. Title: {res_doc['title']}\n   Content: {res_doc['content']}\n")

In [12]:
# search({"query": {"match_all": {}}})
# search({"query": {"match": {"title": "burro e salvia"}}})
# search({"query": {"match": {"content": "banane"}}})
# search({"query": {"multi_match": {"query": "burro", "fields": ["title","content"]}}})
# search({"query": {"match_phrase": {"content": "dalla"}}})

In [14]:
# Ask to user to insert a query
query = input("Enter your search query: ")

# Parsing the user input into a match query for content field
type_query = query.split(":", 1)[0]
content_query = query.split(":", 1)[1].strip()

if type_query == "title":
    if content_query.startswith('"') and content_query.endswith('"'):
        phrase = content_query.strip('"')
        search({"query": {"match_phrase": {"title": phrase}}})
    else:
        search({"query": {"match": {"title": content_query}}})
elif type_query == "content":
    if content_query.startswith('"') and content_query.endswith('"'):
        phrase = content_query.strip('"')
        search({"query": {"match_phrase": {"content": phrase}}})
    else:
        search({"query": {"match": {"content": content_query}}})

Search Results:
1. Title: Carbonara di verdure
   Content: Per preparare la carbonara di verdure mettete a bollire l'acqua per la pasta e a bollore salate. Quindi lavate e asciugate tutti gli ortaggi: dopo aver tagliato via la parte più legnosa dal gambo degli asparagi pelate con un pelapatate, o un coltellino liscio in alternativa, Tagliate anche le punte per il lungo e mettete da parte Spuntate le taccole e poi di nuovo a listarelle sempre per il lungo, come per gli asparagi Stessa cosa per le carote: prima spuntatele, poi pelatele con il pelaverdure o un coltellino infine tegliate a fette e poi a bastoncini sempre per il senso della lunghezza Poi sgranate i piselli Sul fuoco mettete una padella con l’olio e lo spicchio d’aglio mondato intero. Dopo qualche istante, in cui il fondo si sarà riscaldato, potrete tuffare gli asparagi facendoli saltare 2 minuti a fiamma viva Poi unite le taccole e scottate per altri 3 minuti Il fondo si sarà un po’ asciugato a questo punto, perciò aggiunge